In [ ]:
pip install pandas

In [ ]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.4 MB 24.8 MB/s eta 0:00:01
   ------------ --------------------------- 2.8/9.4 MB 29.8 MB/s eta 0:00:01
   -------------------- ------------------- 4.8/9.4 MB 38.1 MB/s eta 0:00:01
   ------------------------------ --------- 7.1/9.4 MB 45.7 MB/s eta 0:00:01
   ---------------------------------- ----- 8.2/9.4 MB 40.3 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.4 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.4 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 30.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.6 kB ? eta -:--:--
   ---------------------------------------- 159.6/159.6 kB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/499.2 kB ? eta -:--:--
   --------------------------------------- 499.2/499.2 kB 30.6 MB/s eta 0:00:00
   ----------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
import random
import itertools
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from logging import raiseExceptions
from random import random

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")
options.add_experimental_option("prefs", prefs)

In [ ]:
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
driver.get('https://www.justicaeleitoral.jus.br/fato-ou-boato/@@pesquisar?SearchableText=&Agencias%3Alist=E-Farsas&periodo_inicial=&periodo_final=#')
lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"']
noticiasTreinamento = []
noticiasValidacao = []
aux = []
tamanhoNoticias = 0

for i in range(3):
    # Acha todos os cards da notícia, faz isso em cada pag
    try:
        newsCards = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located(
                (By.XPATH,
                    '//div[@class="check-labels"]//img[@src="https://www.justicaeleitoral.jus.br/fato-ou-boato/++theme++fatosboatos_v2/images/logos/cards/e-farsas.png"]'
                )
            )
        )
    except:
        raise Exception('elemento não encontrado')

    # Para cada card
    for card in newsCards:
        # Navega até o card e clica
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)
            WebDriverWait(driver, 120).until(EC.element_to_be_clickable(card)).click()
        except ElementClickInterceptedException:
            print("erro")
            continue

        # Muda de janela
        WebDriverWait(driver, 120).until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[-1])

        # Procura a notícia e armazena
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class='wp-caption-text']"))
            )

            newsElement = driver.find_element(By.XPATH, "//p[@class='wp-caption-text']").text
            inicioAspas = newsElement.find('“') + 1
            fimAspas = newsElement.find('”', inicioAspas)
            noticiaLimpa = newsElement[inicioAspas:fimAspas]
            newsElement = noticiaLimpa

            aux = []
            if (random() < 0.8):
                texto = newsElement.strip() # adiciona a noticia
                texto = texto.split()
                for words in texto:
                    for char in lista:
                        words = words.replace(char, '')
                    aux.append(words.lower())

                aux = [palavra for palavra in aux if palavra != '']
                aux.insert(0, 'f')
                noticiasTreinamento.append(aux)
            else:
                texto = newsElement.strip() # adiciona a noticia
                texto = texto.split()
                for words in texto:
                    for char in lista:
                        words = words.replace(char, '')
                    aux.append(words.lower())

                aux = [palavra for palavra in aux if palavra != '']
                aux.insert(0, 'f')
                noticiasValidacao.append(aux)

            tamanhoNoticias += + 1
            print(tamanhoNoticias)

        except Exception as e:
            print("notícias sem dados, pulando...", e)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue


        # Fecha a janela e procura a prox noticia
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    # Aperta o botao de proximo
    next_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="next" and contains(@href, "b_start:int=")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
    next_button.click()

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="check-labels"]'))
    )

    # insert no index 0 f ou r
    # insert um random no 0

1
2
3
4
5
6
notícias sem dados, pulando... Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7d94efe75+79173]
	GetHandleVerifier [0x0x7ff7d94efed0+79264]
	(No symbol) [0x0x7ff7d92a9e5a]
	(No symbol) [0x0x7ff7d9300586]
	(No symbol) [0x0x7ff7d930083c]
	(No symbol) [0x0x7ff7d9354247]
	(No symbol) [0x0x7ff7d93289af]
	(No symbol) [0x0x7ff7d935100d]
	(No symbol) [0x0x7ff7d9328743]
	(No symbol) [0x0x7ff7d92f14c1]
	(No symbol) [0x0x7ff7d92f2253]
	GetHandleVerifier [0x0x7ff7d97ba2ad+3004797]
	GetHandleVerifier [0x0x7ff7d97b46fd+2981325]
	GetHandleVerifier [0x0x7ff7d97d3350+3107360]
	GetHandleVerifier [0x0x7ff7d950a9fe+188622]
	GetHandleVerifier [0x0x7ff7d951228f+219487]
	GetHandleVerifier [0x0x7ff7d94f8dc4+115860]
	GetHandleVerifier [0x0x7ff7d94f8f79+116297]
	GetHandleVerifier [0x0x7ff7d94df528+11256]
	BaseThreadInitThunk [0x0x7ff8758d7ac4+20]
	RtlUserThreadStart [0x0x7ff8785ca4e1+33]

7
8
9
10
11
12
13
14
15
notícias sem dados, pulando... Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7d94efe

KeyboardInterrupt: 

In [ ]:
for i in noticiasTreinamento:
    print(i)

['f', 'só', 'um', 'aviso', 'aqui', 'galera', 'ontem', 'passei', 'pelo', 'treinamento', 'para', 'os', 'trabalhos', 'para', 'a', 'justiça', 'eleitoral', 'no', 'próximo', 'dia', '7', 'lembrem-se', 'de', 'votar', 'em', 'todos', 'os', 'candidatos', 'se', 'votar', 'só', 'em', 'presidente', 'e', 'votar', 'em', 'branco', 'nos', 'outros', 'o', 'voto', 'é', 'tido', 'como', 'voto', 'parcial', 'logo', 'seu', 'voto', 'é', 'anulado', 'só', 'computa', 'voto', 'válido', 'quando', 'o', 'voto', 'é', 'completo', 'questionei', 'isso', 'lá', 'dizendo', 'que', 'a', 'sociedade', 'não', 'tinha', 'ciência', 'de', 'que', 'voto', 'parcial', 'não', 'é', 'computado', 'como', 'voto', 'válido', 'questionei', 'indignado', 'mas', 'a', 'instrutora', 'foi', 'bem', 'clara', 'em', 'dizer', 'que', 'não', 'era', 'computado', 'logo', 'vamos', 'ficar', 'esperto', 'se', 'tiver', 'mais', 'alguém', 'aqui', 'que', 'vá', 'trabalhar', 'e', 'eles', 'tocaram', 'nesse', 'assunto', 'se', 'manifeste']
['f', 'segundo', 'o', 'que', 'diz',

In [ ]:
arqOut = open('./TREINOnoticiasFalsasEFarsas.txt', 'w', encoding='utf-8')
for noticia in noticiasTreinamento:
    print(noticia, file=arqOut)
arqOut.close()

In [ ]:
arqOut = open('./VALIDACAOnoticiasFalsasEfarsas.txt', 'w', encoding='utf-8')
for noticia in noticiasValidacao:
    print(noticia, file=arqOut)
arqOut.close()